In [ ]:
import argparse
import os

from tensorflow.keras.models import load_model

from single_label_network import SingleLabelNetworkTrainer
from train_utils import get_unused_dir_num
from tensorflow.keras.layers import Input

import os
import random

import numpy as np
from imutils import paths
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from train_utils import get_unused_dir_num
from train_utils import get_unused_log_dir_num
from train_utils import load_images
from train_utils import make_logging_callbacks
from train_utils import modify_base_model

from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

import json


In [ ]:

image_width = 28
image_height = 28
batch_size = 32
num_epochs = 100
init_lr = None
train_file_path = 'test_data/vegetables_001/train_list.txt'
val_file_path = None
val_file_path = 'test_data/vegetables_001/val_list.txt'
model_path = 'lenet'
logs_dir = None
full_training = True
resume = False
classes_file_path = os.path.join(os.path.dirname(train_file_path), "classes.txt")
binary_classification = False
if logs_dir is None:
    logs_dir = os.path.join("logs", get_unused_dir_num("logs",train_file_path.split('/')[-2] + '_' + model_path))
os.makedirs(logs_dir, exist_ok=True)
    
class_names = []
with open(classes_file_path) as classes_fp:
    class_names = [line.strip() for line in classes_fp]
    num_classes = len(class_names)
num_classes = len(class_names)
    
# initialize the model
print("[INFO] compiling model...")

if model_path == 'lenet':
    from lenet import LeNet
    base_model = LeNet.build(width=image_width, height=image_height, depth=3, classes=num_classes)
elif model_path == 'mobilenet':
    from tensorflow.keras.applications.mobilenet import MobileNet
    base_model = MobileNet(include_top=True, weights='imagenet')
elif model_path == 'inception_v3':
    from tensorflow.keras.applications.inception_v3 import InceptionV3
    if image_height == image_width:
        base_model = InceptionV3(include_top=True, weights='imagenet')
    else:
        base_model = InceptionV3(include_top=True, weights='imagenet', input_tensor=Input(shape=(image_height, image_width, 3)))
elif model_path == 'EfficientNetB7':
    from tensorflow.keras.applications.efficientnet import EfficientNetB7     
    if image_height == image_width:  
        base_model = EfficientNetB7(include_top=True, weights='imagenet')
    else:
        base_model = EfficientNetB7(include_top=True, weights='imagenet', input_tensor=Input(shape=(image_height, image_width, 3)))
elif model_path == 'NasNet':
    from tensorflow.keras.applications.nasnet import NasNet     
    if image_height == image_width:  
        base_model = NasNet(include_top=True, weights='imagenet')
    else:
        base_model = NasNet(include_top=True, weights='imagenet', input_tensor=Input(shape=(image_height, image_width, 3)))
else:
    base_model = load_model(model_path)

if init_lr is None:
    init_lr = 1e-3 if full_training else 1e-5
# base_model.summary()

In [ ]:

def compile_model(optimizer, num_classes, full_training,
                  resume, binary):
    model = modify_base_model(base_model, 'sigmoid' if num_classes == 1 else 'softmax', num_classes,
                              full_training, resume)
    model.compile(
        optimizer=optimizer,
        loss='mean_squared_error' if binary else 'binary_crossentropy',
        metrics=['accuracy'])
    return model

def load_dataset():
    print("[INFO] loading images...")
    train_paths = []
    train_labels = []

    with open(train_file_path) as train_fp:
        for line in train_fp:
            img_path, class_ids = line.split()

            train_paths.append(img_path)
            train_labels.append(int(class_ids))

    with open(classes_file_path) as classes_fp:
        class_names = [line.strip() for line in classes_fp]
        num_classes = len(class_names)

    with open(os.path.join(logs_dir, "classes.txt"), 'w') as f:
        for item in class_names:
            f.write("%s\n" % item)

    trainX = load_images(train_paths, image_width, image_height)
    train_labels = np.array(train_labels)

    # partition the data into training and testing splits using 75% of
    # the data for training and the remaining 25% for testing
    trainX, testX, trainY, testY = train_test_split(
        trainX, train_labels, test_size=0.25, shuffle=True, random_state=42)
    print(trainY)
    print(testY)

    # convert the labels from integers to vectors
    trainY = to_categorical(trainY, num_classes=num_classes)
    testY = to_categorical(testY, num_classes=num_classes)

    return trainX, testX, trainY, testY, class_names


def load_train_val():
    print("[INFO] loading images...")
    train_paths = []
    train_labels = []
    val_paths = []
    val_labels = []

    with open(train_file_path) as train_fp:
        for line in train_fp:
            img_path, class_ids = line.split()
            train_paths.append(img_path)
            train_labels.append(int(class_ids))

    with open(val_file_path) as val_fp:
        for line in val_fp:
            img_path, class_ids = line.split()
            val_paths.append(img_path)
            val_labels.append(int(class_ids))

    with open(classes_file_path) as classes_fp:
        class_names = [line.strip() for line in classes_fp]
        num_classes = len(class_names)

    with open(os.path.join(logs_dir, "classes.txt"), 'w') as f:
        for item in class_names:
            f.write("%s\n" % item)

    trainX = load_images(train_paths, image_width, image_height)
    trainY = np.array(train_labels)
    testX = load_images(val_paths, image_width, image_height)
    testY = np.array(val_labels)

    # print(trainX)
    # print(testX)
    # print(trainY)
    # print(testY)

    # convert the labels from integers to vectors
    trainY = to_categorical(trainY, num_classes=num_classes)
    testY = to_categorical(testY, num_classes=num_classes)

    return trainX, testX, trainY, testY, class_names

if val_file_path is None:
    x_train, x_test, y_train, y_test, class_names = load_dataset()
else:
    x_train, x_test, y_train, y_test, class_names = load_train_val()

if binary_classification:
    y_train = y_train[:, 1]
    y_test = y_test[:, 1]

num_classes = len(class_names)

# construct the image generator for data augmentation
aug = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest")

optimizer = Adam(lr=init_lr, decay=init_lr / num_epochs)
model = compile_model(optimizer, 1 if binary_classification else num_classes, full_training,
                           resume, binary_classification)

# model.compile(loss="binary_crossentropy",optimizer=opt,
# 	metrics=["accuracy"])
# model.summary()

print(x_train, x_test, y_train, y_test, class_names)


In [ ]:
# print(x_train, y_train)
print(init_lr)
print(len(x_train), len(y_train))
print(len(x_test), len(y_test))
print(len(x_train) // batch_size)

In [33]:

def make_callbacks():
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)
    early_stopping = EarlyStopping(
        monitor='val_loss', patience=100, verbose=1)
#     return [
#         reduce_lr, early_stopping, *make_logging_callbacks(logs_dir)
#     ]
    return [
        *make_logging_callbacks(logs_dir)
    ]
    
    
from lenet import LeNet
base_model = LeNet.build(width=image_width, height=image_height, depth=3, classes=num_classes)

optimizer = Adam(lr=init_lr, decay=init_lr / num_epochs)
model = compile_model(optimizer, 1 if binary_classification else num_classes, full_training,
                           resume, binary_classification)

# train the network
print("[INFO] training network...")
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=batch_size),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // batch_size,
    epochs=num_epochs,
    # validation_steps=800,
    verbose=1,
    callbacks=make_callbacks(),
)

[INFO] training network...
[INFO] serialized model name: logs/vegetables_001_lenet_013model.{epoch:02d}-{val_loss:.2f}.hdf5
Epoch 1/100
1/1 [==============================] - 1s 610ms/step - loss: 0.7119 - accuracy: 0.3750 - val_loss: 0.7090 - val_accuracy: 0.4000

Epoch 00001: val_loss improved from inf to 0.70899, saving model to logs/vegetables_001_lenet_013/model.01-0.71.hdf5
Epoch 2/100
1/1 [==============================] - 0s 119ms/step - loss: 0.7421 - accuracy: 0.1429 - val_loss: 0.6379 - val_accuracy: 0.3333

Epoch 00002: val_loss improved from 0.70899 to 0.63794, saving model to logs/vegetables_001_lenet_013/model.02-0.64.hdf5
Epoch 3/100
1/1 [==============================] - 0s 85ms/step - loss: 0.6409 - accuracy: 0.3438 - val_loss: 0.6426 - val_accuracy: 0.3333

Epoch 00003: val_loss did not improve from 0.63794
Epoch 4/100
1/1 [==============================] - 0s 71ms/step - loss: 0.6380 - accuracy: 0.5714 - val_loss: 0.6534 - val_accuracy: 0.3333

Epoch 00004: val_loss